In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline
import timeit as time
data = loadmat('data/ex3data1.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [2]:
np.set_printoptions(threshold=1000)

X = data['X']
y = data['y']

X.shape, y.shape

((5000, 400), (5000, 1))

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
y_enc=OneHotEncoder(sparse=False).fit_transform(y)
#another_encoder=OneHotEncoder(y).n_values
y_enc.shape

(5000, 10)

In [4]:
y[0], y_enc[0,:]

(array([10], dtype=uint8),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]))

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [273]:
def feedForw(X,theta1,theta2):
    m=X.shape[0]
    
    a1=np.insert(X,0,values=np.ones(m),axis=1)#5000x401
    z2=a1*theta1.T#5000x25
    a2=np.insert(sigmoid(z2),0,values=np.ones(m),axis=1)#5000x26
    z3=a2*theta2.T#5000x10
    h=sigmoid(z3)#5000x10
    
    return a1,z2,a2,z3,h



In [15]:
def cost(params,input_size,hidden_size,num_labels,X,y,learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    a1, z2, a2, z3, h = feedForw(X, theta1, theta2)
    J=np.sum(np.multiply(-y,(np.log(h)))-np.multiply(1-y,np.log(1-h)))/m
    return J

In [525]:
# initial setup
input_size = 400
hidden_size = 25
num_labels = 10
learning_rate = 1
epsilon_init=0.12

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.rand(hidden_size,input_size + 1)*2*epsilon_init-epsilon_init).ravel()
params2= (np.random.rand(num_labels,hidden_size+1)*2*epsilon_init-epsilon_init).ravel()
params=np.concatenate([params,params2])
#params = np.zeros(params.shape)
m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

#theta1.shape, theta2.shape
theta1.shape, theta2.shape, params.shape,params2.shape,


((25, 401), (10, 26), (10285,), (260,))

In [17]:
a1, z2, a2, z3, h = feedForw(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [36]:
cost(params, input_size, hidden_size, num_labels, X, y_enc, learning_rate)

6.9314718055994522

In [537]:
def costReg(params,input_size,hidden_size,num_labels,X,y,learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    lamb=learning_rate
    a1, z2, a2, z3, h = feedForw(X, theta1, theta2)
    Jnr=np.sum(np.multiply(-y,(np.log(h)))-np.multiply(1-y,np.log(1-h)))/m
    reg1=np.sum(np.power(theta1[:,1:],2))
    reg2=np.sum(np.power(theta2[:,1:],2))
    reg=(reg1+reg2)*(float(lamb)/(2*m))
    J=Jnr+reg
    return J
def costReg2(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = feedForw(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    reg1=np.sum(np.power(theta1[:,1:], 2))
    reg2=np.sum(np.power(theta2[:,1:], 2))
    reg=(reg1+reg2)*(float(learning_rate) / (2 * m))
    J += reg
    return J

In [540]:

print(costReg(params, input_size, hidden_size, num_labels, X, y_enc, learning_rate),costReg2(params, input_size, hidden_size, num_labels, X, y_enc, learning_rate))


6.82766943864 6.82766943864


In [23]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [24]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    lamb=learning_rate
    for t in range(m):
        X1=X[t,:]
        y1=y[t,:]
        J=costReg(params,input_size,hidden_size,num_labels,X1,y1,learning_rate)
        a1, z2, a2, z3, h = feedForw(X1, theta1, theta2)
        a3=h#5000x10
        delta3=a3-y1#5000x10
        temp1=np.dot(delta3,theta2)#5000x26
        delta2=np.multiply(temp1[:,1:],sigmoid_gradient(z2))#5000x25
        superDelta2=np.zeros(np.dot(delta3.T,a2).shape)#10x26
        superDelta1=np.zeros(np.dot(delta2.T,a1).shape)#25x401
        superDelta2=superDelta2+np.dot(delta3.T,a2)#10x26
        superDelta1=superDelta1+np.dot(delta2.T,a1)#25x401
        delta1NR=superDelta1/m#25x401
        delta2NR=superDelta2/m#10x26
    delta1=superDelta1/m
    delta2=superDelta2/m
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    return J, grad,delta1,delta2

In [25]:
J,grad,delta1,delta2 = backprop(params, input_size, hidden_size, num_labels, X, y_enc, learning_rate)
J,grad,delta1,delta2
delta1


matrix([[  3.17210305e-06,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -1.57223337e-06,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  1.18366015e-05,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [ -5.47707406e-06,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  7.74871036e-06,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  4.39714307e-06,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [156]:
a1, z2, a2, z3, h = feedForw(X, theta1, theta2)
lamb=1
delta2 = np.zeros(theta1.shape)
delta3 = np.zeros(theta2.shape)#5000x10
J=costReg(params,input_size,hidden_size,num_labels,X,y,learning_rate)
a1, z2, a2, z3, h = feedForw(X, theta1, theta2)
z2t=z2
z2t=np.insert(z2t,0,values=np.ones(z2t.shape[0]),axis=1)
delta3=a3-y
delta2=np.multiply((delta3*theta2),sigmoid_gradient(z2t))
delta2=delta2[:,1:]
Delta1=delta2.T*a1
Delta2=delta3.T*a2

delta3.shape, z2t.shape, z2.shape,delta2.shape, Delta1.shape, Delta2.shape

NameError: name 'a3' is not defined

In [531]:
def backpropReg(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    Delta1 = np.zeros(theta1.shape)
    Delta2 = np.zeros(theta2.shape)
    lamb=learning_rate
    J=0
    a1, z2, a2, z3, h = feedForw(X, theta1, theta2)
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    print(J)
    for t in range(m):
        a1t=a1[t,:]
        z2t=z2[t,:]
        a2t=a2[t,:]
        z3t=z3[t,:]
        yt=y[t,:]
        ht=h[t,:]
        a3t=ht
        delta3=a3t-yt
        z2t = np.insert(z2t, 0, values=np.ones(1))
        temp_delta2=delta3*theta2
        if t==4500:
            print((theta2.T*delta3.T).T==delta3*theta2)
        delta2=np.multiply(temp_delta2,sigmoid_gradient(z2t))
        Delta2=Delta2+delta3.T*a2t
        Delta1=Delta1+(delta2[:,1:]).T*a1t    
    delta1=Delta1/m
    delta2=Delta2/m
    delta1[:,1:]=delta1[:,1:]+(theta1[:,1:]*(lamb/m))
    delta2[:,1:]=delta2[:,1:]+(theta2[:,1:]*(lamb/m))
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    return J, grad


In [532]:
J, grad = backpropReg(params, input_size, hidden_size, num_labels, X, y_enc, learning_rate)
print(J, grad.shape,grad)


6.82766943864
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True]]
6.82766943864 (10285,) [ -7.44962632e-03  -9.47161381e-06   1.60075145e-05 ...,   2.43287227e-01
   1.90520873e-01   2.38401114e-01]


In [539]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backpropReg, x0=params, args=(input_size, hidden_size, num_labels, X, y_enc, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 20})
fmin

6.82766943864
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True]]
6.82766923724
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True]]
6.82766935966
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True]]
3.4323455211
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True]]
3.43234550346
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True]]
3.43234552066
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  Tr

     fun: 2.3661765649924287
     jac: array([  9.18448270e-03,  -9.36032408e-06,   1.58075318e-05, ...,
        -2.32174810e-02,  -4.67194498e-03,  -2.18251397e-02])
 message: 'Max. number of function evaluations reached'
    nfev: 20
     nit: 6
  status: 3
 success: False
       x: array([-0.05564871, -0.04680162,  0.07903766, ..., -0.04678561,
       -0.19522895, -0.1941209 ])

In [510]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = feedForw(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ..., 
       [ 4],
       [ 9],
       [10]], dtype=int64)

In [511]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 89.05999999999999%
